In [ ]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║  Stop-loss-aware Dynamic PIP Trader (drop-in notebook cell)          ║
# ╚══════════════════════════════════════════════════════════════════════╝
import numpy as np
from typing import List, Tuple, Optional

# ────────────────── tiny framework stubs (stand-alone) ──────────────────
def export(fn):        # no-op decorator for back-tester discovery
    fn._is_exported = True
    return fn

class Trader:          # minimal base class
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray: ...
# ───────────────────────── static hyper-params ──────────────────────────
BEST_N: List[int] = [
    200,200,400,200,400,400,100,300,100,200,100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300,
]
BEST_W: List[int] = [
     10, 10,100, 10,100, 10,100, 25, 10,100,100, 50, 25, 10, 50, 25, 10, 25, 50,100,
    100, 10, 10, 50, 25, 25, 10, 50, 25,100, 25, 10,100, 50, 25, 10, 50,100, 50, 25,
     25, 10,100,100, 50, 50, 10, 25, 25, 25,
]
POSITION_LIMIT   = 10_000
THR_MIN, THR_MAX = 0.0, 0.700
THR_STEP         = 0.0005
THR_GRID_FULL    = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=float)
STOP_MULT_GRID   = np.array([0, 1, 2], dtype=int)   # 0 = no stop-loss

# ───────────────────────── helper: causal update ────────────────────────
def causal_update(px: float, last_extreme: float, d: int, thr: float) -> Tuple[int,float]:
    if d == 0:
        m = (px-last_extreme)/last_extreme
        if abs(m) >= thr: return (1 if m>0 else -1), px
        return 0, last_extreme
    if d == 1:
        if px >  last_extreme: return 1, px
        if (last_extreme-px)/last_extreme >= thr: return -1, px
        return 1, last_extreme
    # d == -1
    if px <  last_extreme: return -1, px
    if (px-last_extreme)/last_extreme >= thr: return 1, px
    return -1, last_extreme

# ───────────────────── optimise (thr, stop_mult) on window ──────────────
def best_params(window: np.ndarray,
                prev_thr: Optional[float]) -> Tuple[float,int]:
    # choose candidate threshold grid
    if prev_thr is None:
        cand_thr = THR_GRID_FULL
    else:                         # ±1.5× previous value, clipped
        lo = max(prev_thr - 1.5*prev_thr, THR_MIN)
        hi = min(prev_thr + 1.5*prev_thr, THR_MAX)
        cand_thr = np.linspace(lo, hi, 25)   # fine local grid
    best_thr, best_sm, best_pnl = cand_thr[0], 0, -np.inf
    for thr in cand_thr:
        for sm in STOP_MULT_GRID:
            le=d=0; pos=pnl=0.0
            entry_px=None; pos_dir=0
            for p0,p1 in zip(window[:-1], window[1:]):
                # update direction at bar close p1
                d, le = causal_update(p1, le if le else p0, d, thr)
                # trade logic with stop-loss
                if pos_dir==0 and d!=0:          # new entry
                    entry_px=p1; pos_dir=d
                # stop-loss check
                if pos_dir!=0 and sm>0 and entry_px is not None:
                    if pos_dir== 1 and (p1-entry_px) < -sm*thr*entry_px:
                        pos_dir=0; entry_px=None
                    if pos_dir==-1 and (entry_px-p1) < -sm*thr*entry_px:
                        pos_dir=0; entry_px=None
                pos = pos_dir*POSITION_LIMIT
                pnl += pos*(p1-p0)
            if pnl > best_pnl:
                best_pnl, best_thr, best_sm = pnl, thr, sm
    return best_thr, int(best_sm)

# ───────────────────────────── Trader class ─────────────────────────────
class StopLossPIPTrader(Trader):
    """Dynamic PIP trader with localised threshold grid-search *and*
       integer stop-loss multiplier grid-search."""
    class _S: __slots__=("n","w","next","thr","sm","dir","le","ent")
    def __init__(self):
        self._st=[self._S() for _ in range(50)]
        for s,(n,w) in zip(self._st, zip(BEST_N,BEST_W)):
            s.n,s.w,s.next,s.thr,s.sm,s.dir,s.le,s.ent = n,w,n,None,0,0,None,None
    # ────────────────────────────────────────────────────────────────
    @export
    def Alg(self, P: np.ndarray) -> np.ndarray:  # P shape (50, t_seen)
        n,t = P.shape
        out = np.zeros(n, np.int32)
        for i,s in enumerate(self._st):
            px = P[i,-1]
            if s.le is None: s.le = px
            # ── re-fit when window ready
            if t >= s.next:
                win = P[i, t-s.n:t]
                s.thr, s.sm = best_params(win, s.thr)
                s.next = t + s.w
            if s.thr is None: continue  # still warming up
            # causal direction update
            s.dir, s.le = causal_update(px, s.le, s.dir, s.thr)
            # entry logic
            if out[i]==0 and s.dir!=0:
                s.ent = px; out[i]=s.dir*POSITION_LIMIT
            # stop-loss check
            elif out[i]!=0:
                if s.sm>0 and ((out[i]>0 and (px-s.ent)<-s.sm*s.thr*s.ent) or
                               (out[i]<0 and (s.ent-px)<-s.sm*s.thr*s.ent)):
                    out[i]=0; s.ent=None
                else:
                    out[i]=int(np.sign(out[i]))*POSITION_LIMIT
        return out

# ─────────────────────────── simple back-test ⟡ demo ────────────────────
def _backtest(price_file="prices.txt", steps=None):
    P = np.loadtxt(price_file)          # shape = (T, nInst)
    P = P.T                             # shape = (nInst, T)
    n, T = P.shape
    if steps is None:
        steps = T
    tr = StopLossPIPTrader()
    pos_prev = np.zeros(n, np.int32)
    pnl      = []

    for t in range(1, min(T, steps)):
        # ------------- transpose fixed --------------
        pos = tr.Alg(P[:, :t+1])        # (nInst, t_seen)
        pnl.append((pos_prev * (P[:, t] - P[:, t-1])).sum())
        # --------------------------------------------
        pos_prev = pos

    pnl = np.asarray(pnl, dtype=float)
    print(f"Total PnL : {pnl.sum():,.0f}")
    print(f"Sharpe    : {pnl.mean()/pnl.std(ddof=0)*np.sqrt(252*6.5*60):.2f}")

_backtest()


In [ ]:
#!/usr/bin/env python3
"""
dynamic_pip_trader_hier.py
──────────────────────────
* Same trading rules as the “dynamic-PIP” model you provided.
* Threshold is re-fit with a *hierarchical* coarse→fine search:
      0.00–0.70 @ 0.050  ➜ ±0.050 @ 0.005  ➜ ±0.010 @ 0.001
  → ~8 × fewer PnL evaluations → faster back-tests.
"""
from typing import List, Tuple, Optional
import numpy as np

# ════════════════════════════════════════════════════════════════
#  Infrastructure stubs (leave unchanged in your back-tester)
# ════════════════════════════════════════════════════════════════
def export(fn):                     # no-op decorator
    fn._is_exported = True
    return fn

class Trader:                        # minimal base class
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        raise NotImplementedError

# ════════════════════════════════════════════════════════════════
#  Strategy parameters (unchanged from your last file)
# ════════════════════════════════════════════════════════════════
POSITION_LIMIT = 10_000
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

# threshold search steps
THR_MIN, THR_MAX       = 0.0, 0.700
COARSE_STEP, FINE_STEP, ULTRA_STEP = 0.050, 0.005, 0.001

# ════════════════════════════════════════════════════════════════
#  Helpers
# ════════════════════════════════════════════════════════════════
def causal_update(px: float, le: float, d: int, thr: float) -> Tuple[int, float]:
    """Return updated (direction, last_extreme)."""
    if d == 0:
        mv = (px - le) / le
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, le
    if d == 1:                                 # up-swing
        if px > le:                return 1, px
        if (le - px) / le >= thr:  return -1, px
        return 1, le
    # d == −1
    if px < le:                    return -1, px
    if (px - le) / le >= thr:      return 1, px
    return -1, le


def pnl_for_thr(series: np.ndarray, thr: float) -> float:
    """Net PnL of a single threshold on `series`."""
    d = 0
    le = series[0]
    pnl = 0.0
    for p0, p1 in zip(series[:-1], series[1:]):
        d, le = causal_update(p1, le, d, thr)
        pnl += d * POSITION_LIMIT * (p1 - p0)
    return pnl


def best_threshold_hier(series: np.ndarray) -> float:
    """Hierarchical 3-pass search (coarse → fine → ultra-fine)."""
    # Stage-0: coarse 0.05 grid
    grid = np.arange(THR_MIN, THR_MAX + COARSE_STEP*0.1, COARSE_STEP)
    best = max(grid, key=lambda t: pnl_for_thr(series, t))

    # Stage-1: ±0.05 @ 0.005
    lo = max(THR_MIN, best - 0.05)
    hi = min(THR_MAX, best + 0.05)
    grid = np.arange(lo, hi + FINE_STEP*0.1, FINE_STEP)
    best = max(grid, key=lambda t: pnl_for_thr(series, t))

    # Stage-2: ±0.01 @ 0.001
    lo = max(THR_MIN, best - 0.01)
    hi = min(THR_MAX, best + 0.01)
    grid = np.arange(lo, hi + ULTRA_STEP*0.1, ULTRA_STEP)
    return max(grid, key=lambda t: pnl_for_thr(series, t))

# ════════════════════════════════════════════════════════════════
#  Trader
# ════════════════════════════════════════════════════════════════
class CausalPIPTrader(Trader):

    class _S:  # per-instrument state
        __slots__ = "n", "w", "next", "thr", "dir", "le"
        def __init__(self, n: int, w: int):
            self.n, self.w, self.next = n, w, n
            self.thr = None
            self.dir = 0
            self.le  = None

    def __init__(self):
        if len(BEST_N) != 50 or len(BEST_W) != 50:
            raise ValueError("Need 50 N/W pairs.")
        self._st = [self._S(n, w) for n, w in zip(BEST_N, BEST_W)]
        print("Hier-threshold DynamicPIPTrader ready.")

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        """
        prc shape: (50, t_seen)   – all prices up to *and incl.* current bar.
        Returns int32 vector of positions in shares.
        """
        n_inst, t_seen = prc.shape
        if n_inst != 50:
            raise ValueError("Expected 50 instruments.")
        pos = np.zeros(n_inst, np.int32)

        for i, s in enumerate(self._st):
            px = prc[i, -1]
            if s.le is None:
                s.le = px

            # re-calibrate threshold
            if t_seen >= s.next:
                window = prc[i, t_seen - s.n : t_seen]
                s.thr  = best_threshold_hier(window)
                s.next = t_seen + s.w

            if s.thr is None:
                continue

            s.dir, s.le = causal_update(px, s.le, s.dir, s.thr)
            pos[i] = s.dir * POSITION_LIMIT

        return pos


# ════════════════════════════════════════════════════════════════
#  Quick self-test (remove or comment-out in production)
# ════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    np.random.seed(0)
    P = np.loadtxt("prices.txt")          # shape = (T, nInst)
    P = P.T           
    trader = CausalPIPTrader()
    prev = np.zeros(50, np.int32)
    pnl  = []
    for t in range(1, 1000):
        cur = trader.Alg(P[:, :t+1])
        pnl.append(((prev) * (P[:, t] - P[:, t-1])).sum())
        prev = cur
    pnl = np.asarray(pnl, float)
    sharpe = pnl.mean() / pnl.std(ddof=0) * np.sqrt(252 * 6.5 * 60)
    print(f"Total PnL: {pnl.sum():,.0f}   |   Sharpe: {sharpe:.2f}")


In [ ]:
# Cell 1 – fast trader + helper ------------------------------------------------
"""
fast_dynamic_pip_trader.py
──────────────────────────
• **Same trading logic / outputs** as your reference model
  (rolling N/W re-calibration, ±10 000-share causal-PIP).
• Exhaustive 0.0005-grid search for the best threshold is now **Numba-JIT**
  – identical answer, ~20-40× quicker.
• Ready-to-import `CausalPIPTrader`, decorated with @export.
"""
from __future__ import annotations
from typing import List, Optional, Tuple

import numpy as np
from numba import njit, int8, int32, float64
# ------------------------------------------------------------------
def export(fn): fn._is_exported = True; return fn     # (back-tester stub)
class Trader:
    def Alg(self, prc: np.ndarray) -> np.ndarray:     # (back-tester stub)
        raise NotImplementedError
# ------------------------------------------------------------------
POSITION_LIMIT        = 10_000
THR_MIN, THR_MAX      = 0.0, 0.700
THR_STEP              = 0.0005
THR_GRID              = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

BEST_N: List[int] = [   # 50 values (unchanged)
    200,200,400,200,400,400,100,300,100,200,100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300]
BEST_W: List[int] = [
     10,10,100,10,100,10,100,25,10,100,100,50,25,10,50,25,10,25,50,100,
    100,10,10,50,25,25,10,50,25,100,25,10,100,50,25,10,50,100,50,25,
     25,10,100,100,50,50,10,25,25,25]
# ───────────────────────────── NumPy × Numba core ──────────────────────────
@njit(int8(int8, float64, float64, float64), cache=True, fastmath=True)
def _cu(d, px, le, thr):  # causal_update → direction (int8) | new last_extreme
    if d == 0:
        mv = (px - le) / le
        if abs(mv) >= thr:
            return 1 if mv > 0 else -1
        return 0
    if d == 1:
        if px > le:                         return  1
        if (le - px) / le >= thr:           return -1
        return 1
    # d == −1
    if px < le:                             return -1
    if (px - le) / le >= thr:               return  1
    return -1

@njit(float64(float64[:], float64), cache=True, fastmath=True)
def _pnl_for_thr(series, thr):
    d: int8 = 0
    le = series[0]
    pnl: float64 = 0.0
    for k in range(1, series.size):
        d = _cu(d, series[k], le, thr)
        le = series[k] if (d!=0 and ((d==1 and series[k]>le) or (d==-1 and series[k]<le))) else le
        pnl += d * POSITION_LIMIT * (series[k] - series[k-1])
    return pnl

@njit(float64[:](float64[:]), cache=True, fastmath=True)
def _best_thr(series):
    best_thr = THR_GRID[0]
    best_pnl = -1e100
    for thr in THR_GRID:
        pnl = _pnl_for_thr(series, thr)
        if pnl > best_pnl:
            best_pnl = pnl
            best_thr = thr
    return np.array([best_thr, best_pnl])   # tiny array to return tuple

# ───────────────────────────── Trader class ────────────────────────────────
class CausalPIPTrader(Trader):

    class _S: __slots__ = ("n","w","next","thr","dir","le")
    def __init__(self):
        # build state list
        self._st = []
        for n,w in zip(BEST_N, BEST_W):
            s = self._S()
            s.n, s.w, s.next = n, w, n   # next recal at bar n
            s.thr, s.dir, s.le = None, 0, None
            self._st.append(s)
        print("Fast-JIT DynamicPIPTrader ready.")

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        n_inst, t = prc.shape
        pos = np.zeros(n_inst, np.int32)
        for i, s in enumerate(self._st):
            px = prc[i, -1]
            if s.le is None:                          # first bar
                s.le = px

            if t >= s.next:                           # (re)fit
                win = prc[i, t-s.n : t].astype(np.float64)
                s.thr = _best_thr(win)[0]
                s.next = t + s.w
            if s.thr is None:                         # still flat
                continue
            # causal update
            s.dir = _cu(s.dir, px, s.le, s.thr)
            if (s.dir== 1 and px > s.le) or (s.dir==-1 and px < s.le):
                s.le = px
            pos[i] = s.dir * POSITION_LIMIT
        return pos

# make class importable by back-tester
__all__ = ["CausalPIPTrader"]

# Cell 2 – weakness analyser -------------------------------------------------
"""
Once Cell 1 has run and you have instantiated the fast
`CausalPIPTrader`, execute this cell to:

• run a full back-test on `prices.txt`
• compute per-instrument & per-bar PnL
• show where (volatility quartile, momentum quartile, swing length,
  threshold quartile) the strategy loses money
• list worst instruments & worst 50-bar windows.
"""
import pandas as pd
import matplotlib.pyplot as plt
# Cell 2 – weakness analyser (fixed) ------------------------------------------
"""
After Cell 1 has compiled the fast JIT trader, run this cell
to back-test, bucket PnL, and highlight weak spots.
"""
import pandas as pd
import matplotlib.pyplot as plt

prices = np.loadtxt("prices.txt")          # (1000,50)
prices = prices.T                          # → (50,1000)

# ▶ correct dimension unpacking
nI, T = prices.shape                       # 50 inst, 1000 bars

tr = CausalPIPTrader()
pos_ts = np.zeros((nI, T), np.int32)       # ▶ shape now (50,1000)

prev = np.zeros(nI, np.int32)
pnl  = np.zeros(T)

for t in range(1, T):
    pos = tr.Alg(prices[:, :t+1])
    pos_ts[:, t] = pos                     # ✔ now broadcasts correctly
    pnl[t] = ((prev) * (prices[:, t] - prices[:, t-1])).sum()
    prev   = pos

# ---------------- helper series ----------------
df = pd.DataFrame({
    "bar" : np.arange(T),
    "pnl" : pnl
})

ret  = np.diff(np.log(prices.mean(axis=0)), prepend=np.nan)
df["vol20"] = pd.Series(ret).rolling(20).std()
df["vol_q"] = pd.qcut(df["vol20"], 4, labels=False, duplicates="drop")

mom = (prices.mean(axis=0)[5:] - prices.mean(axis=0)[:-5]) / prices.mean(axis=0)[:-5]
df["mom5"] = pd.Series(np.concatenate((np.full(5, np.nan), mom)))
df["mom_q"] = pd.qcut(df["mom5"], 4, labels=False, duplicates="drop")

def bucket_stats(col):
    g = df.groupby(col)["pnl"]
    return pd.DataFrame({"mean":g.mean(),"sum":g.sum(),"count":g.size()})

print("\n=== PnL by vol20 quartile ===")
print(bucket_stats("vol_q").to_string())
print("\n=== PnL by mom5 quartile ===")
print(bucket_stats("mom_q").to_string())

inst_pnl = (pos_ts[:,:-1] * np.diff(prices)).sum(axis=1)
worst = pd.Series(inst_pnl).nsmallest(5)
print("\n=== Worst 5 instruments ===")
print(worst)

roll50 = pd.Series(pnl).rolling(50).sum()
worst_windows = roll50.nsmallest(5)
print("\n=== Worst 5 rolling-50-bar windows ===")
print(worst_windows)

idx = worst_windows.idxmin()
fig,ax = plt.subplots(figsize=(10,4))
ax.plot(df["bar"], df["pnl"].cumsum(), label="Equity curve")
ax.axvspan(idx-50, idx, color="red", alpha=.3, label="Worst 50-bar")
ax.set_title("Portfolio equity & worst window")
ax.legend()
plt.show()


In [ ]:
# ======================================================================
# Dynamic-PIP Trader – baseline + three targeted improvements
# ======================================================================
import time, itertools, sys
from pathlib import Path
from typing import Tuple, List, Sequence, Optional
import numpy as np

# -------------------- infrastructure stubs (same as back-tester) -------
def export(fn):                       # back-tester decorator – no-op here
    fn._is_exported = True
    return fn
class Trader:                         # minimal interface
    def Alg(self, prc: np.ndarray) -> np.ndarray: ...

# --------------------------- global constants --------------------------
POSITION_LIMIT = 10_000
BEST_N = np.array([
    200,200,400,200,400,400,100,300,100,200,100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300])
BEST_W = np.array([
     10, 10,100, 10,100, 10,100, 25, 10,100,100, 50, 25, 10, 50, 25, 10, 25, 50,100,
    100, 10, 10, 50, 25, 25, 10, 50, 25,100, 25, 10,100, 50, 25, 10, 50,100, 50, 25,
     25, 10,100,100, 50, 50, 10, 25, 25, 25])
THR_MIN, THR_MAX = 0.00, 0.70
THR_COARSE, THR_FINE, THR_ULTRA = 0.05, 0.005, 0.001           # hierarchical grid

# ------------------ helper – causal pivot update -----------------------
@np.vectorize
def _pivot_update(px, le, d, thr):        # vectorized for 1-step speed
    if d==0:
        mv=(px-le)/le
        if abs(mv)>=thr:    return (1 if mv>0 else -1), px
        return 0, le
    if d==1:
        if px>le:           return 1, px
        if (le-px)/le>=thr: return -1, px
        return 1, le
    # d==-1
    if px<le:               return -1, px
    if (px-le)/le>=thr:     return 1, px
    return -1, le

def _pnl(series: np.ndarray, thr: float)->float:
    d=0; le=series[0]; pnl=0.0
    for p0,p1 in zip(series[:-1],series[1:]):
        d,le=_pivot_update(p1,le,d,thr)
        pnl += d*POSITION_LIMIT*(p1-p0)
    return pnl

def _best_thr(series: np.ndarray)->float:
    # hierarchical 3-pass search => ~8× fewer evals than full grid
    g=np.arange(THR_MIN, THR_MAX+THR_COARSE*0.1, THR_COARSE)
    best=max(g, key=lambda t:_pnl(series,t))
    g=np.arange(max(THR_MIN,best-0.05),min(THR_MAX,best+0.05)+THR_FINE*0.1,THR_FINE)
    best=max(g, key=lambda t:_pnl(series,t))
    g=np.arange(max(THR_MIN,best-0.01),min(THR_MAX,best+0.01)+THR_ULTRA*0.1,THR_ULTRA)
    return max(g, key=lambda t:_pnl(series,t))

# ======================================================================
# 1) Base trader – EXACT logic you supplied (fast hierarchical threshold)
# ======================================================================
class BasePIPTrader(Trader):
    class _S: __slots__="n","w","next","thr","dir","le"
    def __init__(self):
        self._st=[self._S() for _ in range(50)]
        for s,n,w in zip(self._st,BEST_N,BEST_W):
            s.n,s.w,s.next=n,w,n; s.thr=None; s.dir=0; s.le=None
    @export
    def Alg(self,prc:np.ndarray)->np.ndarray:
        n,t=prc.shape; pos=np.zeros(n,np.int32)
        for i,s in enumerate(self._st):
            px=prc[i,-1]
            if s.le is None: s.le=px
            if t>=s.next:
                s.thr=_best_thr(prc[i,t-s.n:t]); s.next=t+s.w
            if s.thr is None: continue
            s.dir,s.le=_pivot_update(px,s.le,s.dir,s.thr)
            pos[i]=s.dir*POSITION_LIMIT
        return pos

# ======================================================================
# 2) Variant-A  (VOL-/MOM-filter: skip “chop” bars)
# ======================================================================
def _vol20(arr):                    # unbiased (ddof=0) rolling σ
    return np.sqrt(((arr[-20:]-arr[-20:].mean())**2).mean())
class VolMomFilterTrader(BasePIPTrader):
    def __init__(self,qL,qH): super().__init__(); self.qL,self.qH=qL,qH
    @export
    def Alg(self,prc):                # identical except filter
        n,t=prc.shape; pos=np.zeros(n,np.int32)
        # global momentum & vol for this bar
        mom=(prc[:,-1]-prc[:,-6])/prc[:,-6] if t>5 else np.zeros(n)
        vol=np.array([_vol20(prc[i,:t]) if t>=20 else 0 for i in range(n)])
        mL,mH=np.quantile(mom,[self.qL,self.qH])
        vL,vH=np.quantile(vol,[self.qL,self.qH])
        for i,s in enumerate(self._st):
            px=prc[i,-1]
            if s.le is None: s.le=px
            if t>=s.next:
                s.thr=_best_thr(prc[i,t-s.n:t]); s.next=t+s.w
            if s.thr is None: continue
            # ---------- filter: skip middle buckets --------------
            if (mL<mom[i]<mH) or (vL<vol[i]<vH):
                continue
            s.dir,s.le=_pivot_update(px,s.le,s.dir,s.thr)
            pos[i]=s.dir*POSITION_LIMIT
        return pos

# ======================================================================
# 3) Variant-B  (Instrument down-scaling for worst symbols)
# ======================================================================
WORST = {22,48,6,29,9}           # from diagnostics
class ScaleTrader(BasePIPTrader):
    def __init__(self,scale): super().__init__(); self.scale=scale
    @export
    def Alg(self,prc):
        n,t=prc.shape; pos=np.zeros(n,np.int32)
        for i,s in enumerate(self._st):
            px=prc[i,-1]
            if s.le is None: s.le=px
            if t>=s.next:
                s.thr=_best_thr(prc[i,t-s.n:t]); s.next=t+s.w
            if s.thr is None: continue
            s.dir,s.le=_pivot_update(px,s.le,s.dir,s.thr)
            mult = self.scale if i in WORST else 1.0
            pos[i]=int(s.dir*POSITION_LIMIT*mult)
        return pos

# ======================================================================
# 4) Variant-C  (Adaptive cadence: shorten W when vol spike)
# ======================================================================
VOL_Q = np.quantile         # alias
class AdaptiveCadenceTrader(BasePIPTrader):
    def __init__(self,vol_q): super().__init__(); self.vol_q=vol_q
    @export
    def Alg(self,prc):
        n,t=prc.shape; pos=np.zeros(n,np.int32)
        if t>=20: global_hi = VOL_Q((prc[:,-20:]-prc[:,-20:].mean(axis=1,keepdims=True))**2, self.vol_q, axis=1)
        else:     global_hi = np.full(n, np.inf)
        for i,s in enumerate(self._st):
            px=prc[i,-1]
            if s.le is None: s.le=px
            # shorten calibration cadence during high vol
            if global_hi[i]>0 and t>=20 and np.std(prc[i,-20:])>global_hi[i]:
                eff_w= max(5,s.w//2)
            else:
                eff_w= s.w
            if t>=s.next:
                s.thr=_best_thr(prc[i,max(0,t-s.n):t]); s.next=t+eff_w
            if s.thr is None: continue
            s.dir,s.le=_pivot_update(px,s.le,s.dir,s.thr)
            pos[i]=s.dir*POSITION_LIMIT
        return pos

# ======================================================================
# Back-test harness
# ======================================================================
# ------------------------------------------------------------------
def backtest(prices: np.ndarray, trader_cls, *cls_args):
    n_inst, T = prices.shape         # ― correct order: 50 × T
    tr = trader_cls(*cls_args)
    pnl      = []
    prev_pos = np.zeros(n_inst, np.int32)

    for t in range(1, T):
        # give the trader all history *up to and incl.* bar t
        pos = tr.Alg(prices[:, :t+1])
        pnl.append((prev_pos * (prices[:, t] - prices[:, t-1])).sum())
        prev_pos = pos

    pnl = np.asarray(pnl, float)
    sharpe = 0.0 if pnl.std(ddof=0) == 0 else (
        pnl.mean() / pnl.std(ddof=0) * np.sqrt(252 * 6.5 * 60)
    )
    return pnl.sum(), sharpe, len(pnl)
# ------------------------------------------------------------------


# ======================================================================
# Main driver – small grid-search for each variant
# ======================================================================
if __name__=="__main__":
    prices = np.loadtxt("prices.txt").T          # shape (50, T)
    T = prices.shape[1]
    print(f"Loaded price matrix {T:,d}×50\n")

    results=[]

    # ---------- run baseline ------------------------------------------------
    t0=time.time()
    pnl,sh,nb=backtest(prices,BasePIPTrader)
    results.append(("BASE",pnl,sh,time.time()-t0))

    # ---------- Variant-A grid: (qL,qH) pairs -------------------------------
    for qL,qH in [(0.25,0.75),(0.3,0.7),(0.2,0.8)]:
        tag=f"VOLMOM {qL:.2f}-{qH:.2f}"
        t0=time.time()
        pnl,sh,_=backtest(prices,VolMomFilterTrader,qL,qH)
        results.append((tag,pnl,sh,time.time()-t0))
        print(f"   finished {tag}")

    # ---------- Variant-B grid: scale factors ------------------------------
    for sc in [0.0,0.25,0.5]:
        tag=f"SCALE {sc:.2f}"
        t0=time.time()
        pnl,sh,_=backtest(prices,ScaleTrader,sc)
        results.append((tag,pnl,sh,time.time()-t0))
        print(f"   finished {tag}")

    # ---------- Variant-C grid: high-vol quantile --------------------------
    for vq in [0.8,0.9]:
        tag=f"ADAPT volQ{int(vq*100)}"
        t0=time.time()
        pnl,sh,_=backtest(prices,AdaptiveCadenceTrader,vq)
        results.append((tag,pnl,sh,time.time()-t0))
        print(f"   finished {tag}")

    # ----------------------------------------------------------------------
    print("\n=== Summary (sorted by PnL) ===")
    print(f"{'Model':<15}{'TotalPnL':>12}{'Sharpe':>9}{'Sec':>7}")
    for name,pnl,sh,sec in sorted(results,key=lambda x:-x[1]):
        print(f"{name:<15}{pnl:12,.0f}{sh:9.2f}{sec:7.1f}")


In [ ]:
#!/usr/bin/env python3
"""
per_instrument_gridsearch.py

Grid-search N and W **per instrument** for the dynamic PIP trader,
accounting for commission on dollar turnover.  Prints progress and
a final summary table with best (N, W), total PnL and PnL std for
each of the 50 instruments.
"""

import numpy as np
import pandas as pd
from itertools import product
from pathlib import Path

# ────────────────────────── hyper-parameters ──────────────────────────── #
N_VALUES         = [100, 200, 300, 400]
W_VALUES         = [50, 100, 200]
COMMISSION_RATE  = 0.0005     # 0.05% per dollar traded
POSITION_LIMIT   = 10_000.0
THR_GRID         = np.arange(0.0, 0.700+0.0005, 0.0005)

# ───────────────────────── helper functions ────────────────────────────── #
def causal_update(px: float, le: float, d: int, thr: float):
    if d == 0:
        mv = (px - le) / le
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, le
    if d == 1:
        if px > le:
            return 1, px
        if (le - px) / le >= thr:
            return -1, px
        return 1, le
    # d == -1
    if px < le:
        return -1, px
    if (px - le) / le >= thr:
        return 1, px
    return -1, le

def best_threshold(window: np.ndarray, thr_grid: np.ndarray, position_limit: float) -> float:
    """
    Find threshold that maximizes PnL over `window` using
    the causal_update logic and given position_limit.
    """
    best_thr = thr_grid[0]
    best_pnl = -np.inf
    for thr in thr_grid:
        d, le = 0, window[0]
        pnl = 0.0
        for p0, p1 in zip(window[:-1], window[1:]):
            d, le = causal_update(p1, le, d, thr)
            pnl += d * position_limit * (p1 - p0)
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

def backtest_instrument(series: np.ndarray,
                        N: int,
                        W: int,
                        commission_rate: float,
                        thr_grid: np.ndarray,
                        position_limit: float) -> np.ndarray:
    """
    Back-test one instrument price `series` (length T).
    Re-calibrate threshold every W bars using last Nbars.
    Returns net-PnL time series of length T-1 (commission deducted).
    """
    T = series.size
    pnl = np.zeros(T-1, dtype=float)
    # state
    next_cal = N
    thr      = None
    d        = 0
    le       = series[0]
    prev_pos = 0.0

    for t in range(1, T):
        px = series[t]
        # recalibrate threshold
        if t >= next_cal:
            window = series[max(0, t-N): t]
            thr = best_threshold(window, thr_grid, position_limit)
            next_cal += W

        # decide position
        if thr is not None:
            d, le = causal_update(px, le, d, thr)
        pos = d * position_limit

        # compute bar PnL
        price_diff = px - series[t-1]
        trade_pnl  = prev_pos * price_diff

        # commission on dollar turnover
        turnover   = abs(pos - prev_pos) * px
        commission = commission_rate * turnover

        pnl[t-1]   = trade_pnl - commission
        prev_pos   = pos

    return pnl

# ──────────────────────── main grid-search ───────────────────────────── #
if __name__ == "__main__":
    # load price data: shape (T,50) → transpose to (50,T)
    prices = np.loadtxt(Path("prices.txt"), delimiter=None).T

    results = []
    for inst in range(prices.shape[0]):
        series = prices[inst]
        best_pnl = -np.inf
        best_std = np.nan
        best_N   = None
        best_W   = None

        print(f"\n--- Instrument {inst:02d} ---")
        for N, W in product(N_VALUES, W_VALUES):
            print(f" Testing N={N:3d}, W={W:3d} ...", end="", flush=True)
            pnl_series = backtest_instrument(
                series,
                N,
                W,
                COMMISSION_RATE,
                THR_GRID,
                POSITION_LIMIT
            )
            total_pnl = pnl_series.sum()
            std_pnl   = pnl_series.std(ddof=0)
            print(f" PnL={total_pnl:.2f}, Std={std_pnl:.2f}")

            if total_pnl > best_pnl:
                best_pnl = total_pnl
                best_std = std_pnl
                best_N   = N
                best_W   = W

        print(f" => Best: N={best_N}, W={best_W}, TotalPnL={best_pnl:.2f}, Std={best_std:.2f}")

        results.append({
            "Instrument": inst,
            "Best_N":      best_N,
            "Best_W":      best_W,
            "TotalPnL":    best_pnl,
            "PnL_Std":     best_std
        })

    df = pd.DataFrame(results)
    pd.set_option("display.float_format", "{:.2f}".format)
    print("\n=== Per-Instrument Grid-Search Results ===")
    print(df.to_string(index=False))


In [ ]:
#!/usr/bin/env python3
"""
scale0_report.py
────────────────
▶ Runs the **Scale-0.00** variant of the dynamic-PIP trader  
  (i.e. all positions on the 5 “worst” symbols are forced to 0).  
▶ Computes **per-instrument** statistics:

    • Total net-PnL (after 5 bp commission on $ turn-over)  
    • Std-dev of per-bar net-PnL  

▶ Prints a tidy table.

The trading logic is identical to the code you just tested; only the
reporting layer is different.
"""
from pathlib import Path
import numpy as np
import pandas as pd

# ──────────────────── shared strategy constants ────────────────────── #
POSITION_LIMIT = 10_000
BEST_N = np.array([
    200,200,400,200,400,400,100,300,100,200,100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300])
BEST_W = np.array([
     10, 10,100, 10,100, 10,100, 25, 10,100,100, 50, 25, 10, 50, 25, 10, 25, 50,100,
    100, 10, 10, 50, 25, 25, 10, 50, 25,100, 25, 10,100, 50, 25, 10, 50,100, 50, 25,
     25, 10,100,100, 50, 50, 10, 25, 25, 25])

THR_MIN, THR_MAX              = 0.00, 0.70
THR_COARSE, THR_FINE, THR_ULTRA = 0.05, 0.005, 0.001
WORST = {22, 48, 6, 29, 9}          # symbols to turn off
COMM_RATE = 0.0005                  # 5 bp

# ─────────────────── helper – causal PIP bits ───────────────────────── #
def pivot_update(px, le, d, thr):
    if d == 0:
        mv = (px - le) / le
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, le
    if d == 1:
        if px > le:
            return 1, px
        if (le - px) / le >= thr:
            return -1, px
        return 1, le
    # d == −1
    if px < le:
        return -1, px
    if (px - le) / le >= thr:
        return 1, px
    return -1, le


def pnl_for_thr(series: np.ndarray, thr: float) -> float:
    d = 0
    le = series[0]
    pnl = 0.0
    for p0, p1 in zip(series[:-1], series[1:]):
        d, le = pivot_update(p1, le, d, thr)
        pnl += d * POSITION_LIMIT * (p1 - p0)
    return pnl


def best_thr(series: np.ndarray) -> float:
    # 3-level hierarchical search
    grid = np.arange(THR_MIN, THR_MAX + THR_COARSE * 0.1, THR_COARSE)
    best = max(grid, key=lambda t: pnl_for_thr(series, t))

    lo, hi = max(THR_MIN, best - 0.05), min(THR_MAX, best + 0.05)
    grid   = np.arange(lo, hi + THR_FINE * 0.1, THR_FINE)
    best   = max(grid, key=lambda t: pnl_for_thr(series, t))

    lo, hi = max(THR_MIN, best - 0.01), min(THR_MAX, best + 0.01)
    grid   = np.arange(lo, hi + THR_ULTRA * 0.1, THR_ULTRA)
    return max(grid, key=lambda t: pnl_for_thr(series, t))


# ───────────────────── Scale-0 trader class ─────────────────────────── #
class Scale0Trader:
    class _S: __slots__ = "n", "w", "next", "thr", "dir", "le"
    def __init__(self):
        self.state = []
        for n, w in zip(BEST_N, BEST_W):
            s = self._S()
            s.n, s.w, s.next = int(n), int(w), int(n)
            s.thr = None
            s.dir = 0
            s.le  = None
            self.state.append(s)

    def Alg(self, prc: np.ndarray) -> np.ndarray:
        """
        prc shape → (50, t_seen).  Returns int32 positions.
        """
        n_inst, t = prc.shape
        pos = np.zeros(n_inst, np.int32)

        for i, s in enumerate(self.state):
            px = prc[i, -1]
            if s.le is None:
                s.le = px

            # re-fit threshold
            if t >= s.next:
                window = prc[i, max(0, t - s.n): t]
                s.thr = best_thr(window)
                s.next = t + s.w

            if s.thr is None:            # still warming up
                continue

            s.dir, s.le = pivot_update(px, s.le, s.dir, s.thr)

            if i in WORST:               # scale == 0 → flat
                pos[i] = 0
            else:
                pos[i] = s.dir * POSITION_LIMIT
        return pos


# ───────────────────── back-test + per-inst stats ────────────────────── #
def backtest_per_instrument(prices: np.ndarray):
    """
    prices shape (50, T).  Returns DataFrame with PnL totals & std-devs.
    """
    n_inst, T = prices.shape
    trader = Scale0Trader()

    pnl_matrix = np.zeros((n_inst, T - 1), dtype=float)
    prev_pos   = np.zeros(n_inst, np.int32)

    for t in range(1, T):
        pos = trader.Alg(prices[:, : t + 1])
        price_diff = prices[:, t] - prices[:, t - 1]

        # commission on turnover
        turnover = np.abs(pos - prev_pos) * prices[:, t]
        commission = COMM_RATE * turnover

        pnl_matrix[:, t - 1] = prev_pos * price_diff - commission
        prev_pos = pos

    df = pd.DataFrame({
        "Instrument": np.arange(n_inst),
        "TotalPnL":   pnl_matrix.sum(axis=1),
        "StdPnL":     pnl_matrix.std(axis=1, ddof=0),
    })
    return df


# ─────────────────────────────── main ────────────────────────────────── #
if __name__ == "__main__":
    prices = np.loadtxt(Path("prices.txt"), delimiter=None).T  # (50, T)

    df_stats = backtest_per_instrument(prices)

    pd.set_option("display.float_format", "{:.2f}".format)
    print("\n=== Scale-0.00 Trader – per-instrument stats ===")
    print(df_stats.to_string(index=False))

    print("\nPortfolio total PnL : {:,.0f}".format(df_stats.TotalPnL.sum()))
    print("Portfolio PnL σ     : {:.2f}".format(df_stats.StdPnL.sum()))


In [ ]:
from pathlib import Path
from typing    import List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ──────────────────── Trader scaffolding ─────────────────────
def export(fn):
    """No-op decorator to mark exported methods."""
    fn._is_exported = True
    return fn

class Trader:
    """Minimal base class for our model."""
    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        raise NotImplementedError

# ────────────────────── strategy hyper‐parameters ───────────────────
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT   = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID         = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=float)
COMMISSION_RATE  = 0.0005  # per dollar traded

# ────────────────────── helper functions ─────────────────────────
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    """
    One‐step causal PIP update.
    Returns (new_direction, new_last_extreme).
    """
    if direction == 0 or direction is None:
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """Grid‐search optimal threshold on `window`."""
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), dtype=np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) * POSITION_LIMIT * np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

# ───────────────────────── trader class ───────────────────────────
class CausalPIPTrader(Trader):
    """Per‐instrument PIP trader producing positions and confidences."""
    class _InstrState:
        __slots__ = ("n","w","next_calib","thr","dir","last_extreme")
        def __init__(self,n:int,w:int):
            self.n=n; self.w=w; self.next_calib=n; self.thr=None; self.dir=0; self.last_extreme=None

    def __init__(self):
        super().__init__()
        self._states = [self._InstrState(n,w) for n,w in zip(BEST_N,BEST_W)]
        print("CausalPIPTrader initialized.")

    @export
    def Alg(self,prcSoFar:np.ndarray)->Tuple[np.ndarray,np.ndarray]:
        nInst,t_seen = prcSoFar.shape
        positions = np.zeros(nInst,dtype=np.int32)
        confidences = np.zeros(nInst,dtype=float)
        for i in range(nInst):
            st=self._states[i]; price=prcSoFar[i,-1]
            if st.last_extreme is None: st.last_extreme=price
            if t_seen>=st.next_calib:
                window=prcSoFar[i,t_seen-st.n:t_seen]
                st.thr=best_threshold(window)
                st.next_calib+=st.w
                print(f"[t={t_seen}] Inst {i} thr={st.thr:.4f}")
            if st.thr is None: continue
            prev_ext=st.last_extreme
            new_dir,new_ext=causal_update(price,prev_ext,st.dir,st.thr)
            move_ratio=abs((price-prev_ext)/prev_ext)/st.thr if st.thr>0 else 0.0
            confidences[i]=min(move_ratio,1.0)
            st.dir,st.last_extreme=new_dir,new_ext
            positions[i]=new_dir*POSITION_LIMIT
        return positions,confidences

prices = np.loadtxt(Path("prices.txt"))  # shape (T,50)
T, nInst = prices.shape
trader = CausalPIPTrader()
pos_arr = np.zeros((T, nInst), dtype=int)
conf_arr = np.zeros((T, nInst), dtype=float)
for t in range(1, T+1):
    pos_arr[t-1], conf_arr[t-1] = trader.Alg(prices[:t, :].T)

# ─── Compute PnL breakdown across time ───────────────────────────
trade_pnl = np.zeros((T, nInst), dtype=float)
comm_pnl  = np.zeros((T, nInst), dtype=float)
for t in range(1, T):
    prev, curr = pos_arr[t-1], pos_arr[t]
    diff = prices[t] - prices[t-1]
    trade_pnl[t] = prev * diff
    comm_pnl[t]  = -COMMISSION_RATE * np.abs(curr - prev) * prices[t]

pnl_total = trade_pnl + comm_pnl
cum_pnl    = np.cumsum(pnl_total, axis=0)

# ─── Identify largest drawdowns per instrument ───────────────────
results = []
for inst in range(nInst):
    # per‐step change in cum‐PnL
    delta = np.diff(cum_pnl[:, inst], prepend=0)
    drops = np.where(delta < 0)[0]
    for t in drops:
        results.append((inst, t, delta[t], trade_pnl[t, inst], comm_pnl[t, inst]))

df = pd.DataFrame(results, columns=["inst", "t", "drop", "trade_pnl", "comm_pnl"])

# **Fixed selection**: sort by drop, then take top 5 per instrument
top5_sorted = (
    df
      .sort_values("drop")           # most negative first
      .groupby("inst", as_index=False)
      .head(5)                       # first 5 rows per instrument
      .sort_values(["inst", "drop"])
)

print("Top 5 largest single-step drawdowns per instrument:")
print(top5_sorted)

# ─── plotting per instrument ───────────────────────────────────────
for inst in range(nInst):
    times = np.arange(T)
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10,12), sharex=True)

    # 1) Price + shading
    ax1.plot(times, prices[:, inst], color="black")
    ax1.fill_between(times, prices[:, inst].min(), prices[:, inst],
                     where=pos_arr[:, inst] > 0, facecolor="green", alpha=0.3, step="mid")
    ax1.fill_between(times, prices[:, inst].max(), prices[:, inst],
                     where=pos_arr[:, inst] < 0, facecolor="red", alpha=0.3, step="mid")
    ax1.set_ylabel("Price")

    # 2) Cumulative PnL
    ax2.plot(times, cum_pnl[:, inst], linewidth=1)
    ax2.axhline(0, color="black", linewidth=0.5)
    ax2.set_ylabel("Cumulative PnL")

    # 3) Confidence
    ax3.plot(times, conf_arr[:, inst], linewidth=1)
    ax3.set_ylim(0, 1)
    ax3.set_ylabel("Confidence")
    ax3.set_xlabel("Time")

    fig.suptitle(f"Instrument {inst:02d}")
    plt.tight_layout(rect=[0,0.03,1,0.95])
    plt.show()

In [ ]:
#!/usr/bin/env python3
"""
observer_pip_trader.py
──────────────────────
Dynamic-PIP strategy that *self-monitors* its recent virtual PnL:

    • For every instrument we keep running “virtual” PnL as if we
      traded with full size, even when we are actually flat.
    • The model becomes **active** (takes real positions) only when
      the virtual PnL has been in a clear up-trend:
            cumPnL_last_WIN  –  cumPnL_prev_win ≥ +THRESH.
    • It *de-activates* (goes flat) once the same metric drops below
      −THRESH (edge lost / draw-down starting).

Default: WIN = 100 bars, THRESH = 1 × POSITION_LIMIT dollars.  
All other mechanics (N/W grid, hierarchical threshold fit, causal
 updates, 5 bp commission) are unchanged.

The script prints an **ACTIVATE / DEACTIVATE** message with the bar
index and instrument id whenever a regime switch happens.
"""
from pathlib import Path
from typing  import List, Optional, Tuple
import numpy as np

# ─────────────────── strategy constants ──────────────────────
POSITION_LIMIT      = 10_000
ACTIVATE_WIN        = 100          # look-back window
ACTIVATE_THRESH     = 1 * POSITION_LIMIT   # $ threshold

BEST_N = np.array([
    200,200,400,200,400,400,100,300,100,200,100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300])

BEST_W = np.array([
     10, 10,100, 10,100, 10,100, 25, 10,100,100, 50, 25, 10, 50, 25, 10, 25, 50,100,
    100, 10, 10, 50, 25, 25, 10, 50, 25,100, 25, 10,100, 50, 25, 10, 50,100, 50, 25,
     25, 10,100,100, 50, 50, 10, 25, 25, 25])

THR_MIN, THR_MAX            = 0.00, 0.70
THR_COARSE, THR_FINE, THR_ULTRA = 0.05, 0.005, 0.001
COMM_RATE                   = 0.0005   # 5 bp


# ─────────────────── helper functions ────────────────────────
def pivot_update(px, le, d, thr):
    if d == 0:
        mv = (px - le) / le
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, le
    if d == 1:
        if px > le:  return 1, px
        if (le - px) / le >= thr: return -1, px
        return 1, le
    if px < le:       return -1, px
    if (px - le) / le >= thr: return 1, px
    return -1, le


def pnl_for_thr(series: np.ndarray, thr: float) -> float:
    d = 0; le = series[0]; pnl = 0.0
    for p0, p1 in zip(series[:-1], series[1:]):
        d, le = pivot_update(p1, le, d, thr)
        pnl += d * POSITION_LIMIT * (p1 - p0)
    return pnl


def best_thr(series: np.ndarray) -> float:
    g = np.arange(THR_MIN, THR_MAX + THR_COARSE*0.1, THR_COARSE)
    best = max(g, key=lambda t: pnl_for_thr(series, t))
    g = np.arange(max(THR_MIN, best-0.05), min(THR_MAX, best+0.05)+THR_FINE*0.1, THR_FINE)
    best = max(g, key=lambda t: pnl_for_thr(series, t))
    g = np.arange(max(THR_MIN, best-0.01), min(THR_MAX, best+0.01)+THR_ULTRA*0.1, THR_ULTRA)
    return max(g, key=lambda t: pnl_for_thr(series, t))


# ─────────────────── Trader with self-monitor ──────────────────────────
def export(fn): fn._is_exported = True; return fn   # back-tester stub
class Trader:  # minimal base
    def Alg(self, prc): raise NotImplementedError


class ObserverPIPTrader(Trader):
    class _S:
        __slots__ = ("n","w","next","thr","dir","le",
                     "virt_pnl","cum_pnl","active")
        def __init__(self, n:int, w:int):
            self.n, self.w, self.next = n, w, n
            self.thr = None
            self.dir = 0
            self.le  = None
            self.virt_pnl = []            # last ACTIVATE_WIN virtual pnl values
            self.cum_pnl  = 0.0
            self.active   = False         # starts inactive

    def __init__(self):
        if len(BEST_N)!=50: raise ValueError("Need 50 N/W pairs")
        self.state = [self._S(int(n), int(w)) for n, w in zip(BEST_N, BEST_W)]
        print("ObserverPIPTrader ready.")

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        """
        prc shape  (50, t_seen)  – returns int32 positions in shares.
        """
        n, t = prc.shape
        pos = np.zeros(n, np.int32)

        for i, s in enumerate(self.state):
            px = prc[i,-1]
            if s.le is None: s.le = px

            # (re)fit threshold
            if t >= s.next:
                s.thr = best_thr(prc[i, max(0, t-s.n): t])
                s.next = t + s.w

            # virtual direction for performance probing
            if s.thr is not None:
                virt_dir, s.le = pivot_update(px, s.le, s.dir, s.thr)
            else:
                virt_dir = 0

            # ---------- virtual PnL update --------------------------------
            if t > 1:
                virt_pnl_t = virt_dir * POSITION_LIMIT * (px - prc[i, -2])
            else:
                virt_pnl_t = 0.0
            s.cum_pnl += virt_pnl_t
            s.virt_pnl.append(virt_pnl_t)
            if len(s.virt_pnl) > ACTIVATE_WIN:
                s.cum_pnl -= s.virt_pnl.pop(0)

            # ---------- regime switching ----------------------------------
            if not s.active and len(s.virt_pnl) == ACTIVATE_WIN:
                if s.cum_pnl >= ACTIVATE_THRESH:
                    s.active = True
                    print(f"[t={t}] Inst {i:02d} ACTIVATED (virtPnL={s.cum_pnl:.0f})")
            elif s.active:
                if s.cum_pnl <= -ACTIVATE_THRESH:
                    s.active = False
                    print(f"[t={t}] Inst {i:02d} DEACTIVATED (virtPnL={s.cum_pnl:.0f})")

            # ---------- real trading position -----------------------------
            if s.active and s.thr is not None:
                s.dir = virt_dir          # adopt virtual signal
                pos[i] = s.dir * POSITION_LIMIT
            else:
                s.dir = virt_dir          # keep updating dir for probes
                pos[i] = 0

        return pos


# ─────────────────── simple demo / sanity check ────────────────────────
if __name__ == "__main__":
    prices = np.loadtxt(Path("prices.txt"), delimiter=None).T  # (50, T)
    trader = ObserverPIPTrader()
    T = prices.shape[1]
    prev = np.zeros(50, np.int32); pnl=[]
    for t in range(1, T):
        cur = trader.Alg(prices[:, :t+1])
        pnl.append((prev*(prices[:,t]-prices[:,t-1])).sum()
                   - COMM_RATE * np.abs(cur-prev) @ prices[:,t])
        prev = cur
    pnl = np.array(pnl,float)
    print(f"\nTotal net-PnL : {pnl.sum():,.0f}")
    if pnl.std(ddof=0)>0:
        sh = pnl.mean()/pnl.std(ddof=0)*np.sqrt(252*6.5*60)
        print(f"Sharpe ratio  : {sh:.2f}")
